In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import pydicom
import os
from tqdm import tqdm

In [2]:
os.listdir('../input/siim-acr-pneumothorax-segmentation/stage_2_images')

['ID_58fc524c7.dcm',
 'ID_399d97c02.dcm',
 'ID_ab50a8ac6.dcm',
 'ID_cfa0b2d68.dcm',
 'ID_3b83260ac.dcm',
 'ID_44e8b2bbe.dcm',
 'ID_69cc2783a.dcm',
 'ID_c9636e74d.dcm',
 'ID_9aefb2452.dcm',
 'ID_49978f1d6.dcm',
 'ID_89770c8a9.dcm',
 'ID_ccc615b5f.dcm',
 'ID_422f28eef.dcm',
 'ID_854c6545e.dcm',
 'ID_51b713bb5.dcm',
 'ID_7fbe933ef.dcm',
 'ID_73c832baa.dcm',
 'ID_b7c364a30.dcm',
 'ID_6c32ef1b6.dcm',
 'ID_609806ebc.dcm',
 'ID_4efe34798.dcm',
 'ID_dfda795f0.dcm',
 'ID_5f5d5104b.dcm',
 'ID_899ddca20.dcm',
 'ID_6700074e3.dcm',
 'ID_0359f0e2c.dcm',
 'ID_ea2d1f89f.dcm',
 'ID_3094abf9e.dcm',
 'ID_98e39f707.dcm',
 'ID_7501d209f.dcm',
 'ID_66132c9be.dcm',
 'ID_ea252d738.dcm',
 'ID_b92b8b0a2.dcm',
 'ID_26bc2f4d1.dcm',
 'ID_de0ef3c02.dcm',
 'ID_059491a97.dcm',
 'ID_7f143dc32.dcm',
 'ID_8338fb7eb.dcm',
 'ID_41cd23d84.dcm',
 'ID_206db31d3.dcm',
 'ID_bd977da32.dcm',
 'ID_7f01c10e0.dcm',
 'ID_4b28feec0.dcm',
 'ID_882796178.dcm',
 'ID_435111c89.dcm',
 'ID_113701c77.dcm',
 'ID_cb071202a.dcm',
 'ID_03f956d9

In [3]:
df_samp_sub = pd.read_csv('../input/siim-acr-pneumothorax-segmentation/stage_2_sample_submission.csv')
df_samp_sub.head()

,ImageId,EncodedPixels
0,ID_0011fe81e,-1
1,ID_9ca06e9b9,-1
2,ID_6f00d6ce6,-1
3,ID_9258110b0,-1
4,ID_fa01c9546,-1


In [4]:
from glob import glob
test_fns = sorted(glob('../input/siim-acr-pneumothorax-segmentation/stage_2_images/*.dcm'))

Utility functions

In [5]:
def run_length_encode(component):
    component = component.T.flatten()
    start = np.where(component[1:] > component[:-1])[0]+1
    end = np.where(component[:-1] > component[1:])[0]+1
    length = end-start
    rle = []
    for i in range(len(length)):
        if i == 0:
            rle.extend([start[0], length[0]])
        else:
            rle.extend([start[i]-end[i-1], length[i]])
    rle = ' '.join([str(r) for r in rle])
    return rle

Load relevant libraries for the models

In [6]:
import torchvision

In [7]:
import os
import cv2
import pdb
import time
import warnings
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, sampler
from matplotlib import pyplot as plt
from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.torch import ToTensor
!pip install git+https://github.com/qubvel/segmentation_models.pytorch > /dev/null 2>&1 # run bash cmd without output.
import segmentation_models_pytorch as smp
warnings.filterwarnings("ignore")

Create the dataloader

In [8]:
class TestDataset(Dataset):
    def __init__(self,df_samp_sub, size, mean, std, tta=4):
        
        self.size = size
        self.fnames = list(df_samp_sub["ImageId"])
        self.num_samples = len(self.fnames)
        self.transform = Compose([Normalize(mean=mean, std=std, p=1),Resize(size, size),ToTensor(),])

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = '../input/siim-acr-pneumothorax-segmentation/stage_2_images/' + fname + '.dcm'
        dset = pydicom.dcmread(path)
        img = dset.pixel_array
        img = np.expand_dims(img,axis = 2)
        b = img.copy()
        c = img.copy()
        img = np.concatenate((img,b,c),axis = 2)
        augmented = self.transform(image = img)
        img = augmented['image']
        return(img) 
        

    def __len__(self):
        return self.num_samples

In [9]:
# Create dataloader for testset
size = 512
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
num_workers = 8
batch_size = 4
best_threshold = 0.5
min_size = 3500
device = torch.device("cuda:0")
testset = DataLoader(TestDataset(df_samp_sub,size, mean, std),batch_size=batch_size,shuffle=False,num_workers=num_workers,pin_memory=True,)
encoded_pixels = []

Post-process utility functions

In [10]:
def post_process(probability, threshold, min_size):
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((1024, 1024), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

Load models

In [11]:
model_resnet = smp.Unet("resnet34", encoder_weights="imagenet", activation=None).cuda()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /tmp/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:03<00:00, 28.4MB/s]


In [12]:
path = '../input/model-resnet34-3/model_resnet34_3.pth'
x = torch.load(path)
model_resnet.load_state_dict(x['state_dict'])
model_resnet.eval()
#model_unet_resnet.train()

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

Loading Chexnet - Requires a little more work since the state_dict keys dont match initially. 

In [13]:
class DenseNet121(nn.Module):

    def __init__(self, classCount, isTrained):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=isTrained)
        kernelCount = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(nn.Linear(kernelCount, classCount), nn.Sigmoid())

    def forward(self, x):
        x = self.densenet121(x)
        return x

In [14]:
# Need to correct keys of state_dict
from collections import OrderedDict
e = [str(i) for i in range(10)]
new_dict = OrderedDict()
for k,v in y['state_dict'].items():
    b = k.split('.')
    if b[-2] in e:
        b[-3] = b[-3]+b[-2]
        del(b[-2])
        s = '.'.join(b)
        new_dict[s] = v
    else:
        new_dict[k] = v 
new_dict['module.densenet121.classifier.0.weight'] = new_dict.pop('module.densenet121.classifier0.weight',None) 
new_dict['module.densenet121.classifier.0.bias'] = new_dict.pop('module.densenet121.classifier0.bias',None)

NameError: name 'y' is not defined

In [15]:
# Load Chexnet 
path1 = '../input/chexnet/repository/zoogzog-chexnet-bbe30b5/models/m-25012018-123527.pth.tar'
model2 = DenseNet121(14, True).cuda()
model2 = torch.nn.DataParallel(model2).cuda()
modelCheckpoint = torch.load(path1)
model2.load_state_dict(new_dict)
model2.eval()

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /tmp/.cache/torch/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:01<00:00, 30.5MB/s]


RuntimeError: Error(s) in loading state_dict for DataParallel:
	Missing key(s) in state_dict: "module.densenet121.features.conv0.weight", "module.densenet121.features.norm0.weight", "module.densenet121.features.norm0.bias", "module.densenet121.features.norm0.running_mean", "module.densenet121.features.norm0.running_var", "module.densenet121.features.denseblock1.denselayer1.norm1.weight", "module.densenet121.features.denseblock1.denselayer1.norm1.bias", "module.densenet121.features.denseblock1.denselayer1.norm1.running_mean", "module.densenet121.features.denseblock1.denselayer1.norm1.running_var", "module.densenet121.features.denseblock1.denselayer1.conv1.weight", "module.densenet121.features.denseblock1.denselayer1.norm2.weight", "module.densenet121.features.denseblock1.denselayer1.norm2.bias", "module.densenet121.features.denseblock1.denselayer1.norm2.running_mean", "module.densenet121.features.denseblock1.denselayer1.norm2.running_var", "module.densenet121.features.denseblock1.densel

Generate a sample submission

In [16]:
encoded_pixels = []
for i, batch in enumerate(tqdm(testset)):
    a = model2(batch)
    b = a[:,7]
    preds = torch.sigmoid(model_resnet(batch.to(device)))
    preds = preds.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
    for i,probability in enumerate(preds):
        if probability.shape != (1024, 1024):
            probability = cv2.resize(probability, dsize=(1024, 1024), interpolation=cv2.INTER_LINEAR)
        predict, num_predict = post_process(probability, best_threshold, min_size)
        if num_predict == 0 or b[i] <= .09:
            encoded_pixels.append('-1')
        else:
            r = run_length_encode(predict)
            encoded_pixels.append(r)

In [17]:
df_samp_sub['EncodedPixels'] = encoded_pixels

In [18]:
df_samp_sub.to_csv('submission.csv', columns=['ImageId', 'EncodedPixels'], index=False)